# CNN GAN Training

In [1]:
cd ../..

/Users/hao/workspace/hpi_de/4th_Semester/Applied Machine Learning/sensor-data-gans


## imports

In [2]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from numpy.random import randn
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv1D, Conv1DTranspose, MaxPool1D, GlobalMaxPool1D, Flatten, Dropout, LeakyReLU, Reshape, BatchNormalization
from tensorflow.keras.optimizers import Adam
from gans.gan_trainer import GanTrainer

## Create Discriminator and Generator Networks
methods which creates and returns generator and discriminator

In [3]:
def create_discriminator(input_shape, optimizer=Adam(learning_rate=0.0001, clipvalue=1.0, decay=1e-8)):
    discriminator = Sequential()
    discriminator.add(Conv1D(8, kernel_size=9, activation='relu', input_shape=input_shape))
    discriminator.add(Flatten())
    discriminator.add(Dense(1, activation='sigmoid'))

    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return discriminator

In [4]:
def create_generator(latent_dim, kernel_size, kernel_num=3, small=True):
    generator = Sequential()
    generator.add(Dense(125 * 3, kernel_initializer='he_uniform', input_shape=(latent_dim,)))
    generator.add(LeakyReLU())
    generator.add(BatchNormalization())

    generator.add(Reshape((125, 3)))

    generator.add(Conv1D(kernel_num, kernel_size=kernel_size, kernel_initializer='he_uniform', padding='same'))
    generator.add(LeakyReLU())
    generator.add(BatchNormalization())

    generator.add(Conv1DTranspose(kernel_num, kernel_size=4, strides=2, padding='same'))
    generator.add(LeakyReLU())
    generator.add(BatchNormalization())

    generator.add(Conv1D(3, kernel_size=kernel_size, padding='same', kernel_initializer='he_uniform', activation='linear'))
    
    return generator

## Training Routine

### parameters for data loading and preprocessing

In [5]:
# data parameters
train_path = './datasets/mydata/train_df.h5'
val_path = './datasets/mydata/val_df.h5'
test_path = './datasets/mydata/test_df.h5'

# what action to generate
act_id = 0

# preprocessing parameters
window_size = 5*50
step_size = int(window_size/2)
method ='sliding'
col_names = ['userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z', 'userAcceleration.c']
input_cols_train=['userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z']
input_cols_eval=['userAcceleration.c']

### parameters for training

In [6]:
steps = 1000
batch_size = 32
eval_step = 100
random = False
label_smoothing = False

## Training routines with different hyper parameters

In [ ]:
for i in range(3): # repeats
    for act_id in range(0,1): # for each action
        for latent_dim in [32, 64, 128, 256]: # different latent dimensions
            for kernel_size in [3, 5, 7, 9, 11, 13, 15]: # different kernel sizes
                
                out_dir = './out/cnn/act_id-{}/{}_ld-{}_ks-{}'.format(act_id, datetime.now(), latent_dim, kernel_size)
                print("New: ", out_dir, 'Act_id:', act_id)
                
                gan_trainer = GanTrainer(out_dir=out_dir)
                gan_trainer.init_data(train_path=train_path,
                      val_path=val_path,
                      test_path=test_path,
                      act_id=act_id,
                      window_size=window_size,
                      step_size=step_size,
                      method=method,
                      col_names=col_names,
                      input_cols_train=input_cols_train,
                      input_cols_eval=input_cols_eval
                     )
                
                input_shape = gan_trainer.x_train[0].shape

                discriminator = create_discriminator(input_shape)
                generator = create_generator(latent_dim, kernel_size=kernel_size)
                gan_trainer.create_gan(generator=generator, discriminator=discriminator)

                gan_trainer.train_gan(steps=steps, 
                          batch_size=batch_size, 
                          eval_step=eval_step,
                          random=random, 
                          label_smoothing=label_smoothing)

New:  ./out/cnn/act_id-0/2020-08-15 16:52:41.658018_ld-32_ks-3 Act_id: 0
Load Data...
Transform Data...
Calculate origin performance...
Done!
discriminator loss: [0.6731553673744202, 0.4375]
adversarial loss: 1.1896641254425049


Train f1-score for act_id  0 improved from  0.0 to  0.7412553905126976
Validation f1-score for act_id  0 improved from  0.0 to  0.7369323050556984
discriminator loss: [0.48379260301589966, 0.75]
adversarial loss: 2.479581356048584


discriminator loss: [0.48690301179885864, 0.5625]
adversarial loss: 1.7975653409957886


Train f1-score for act_id  0 improved from  0.7412553905126976 to  0.749515503875969
Validation f1-score for act_id  0 improved from  0.7369323050556984 to  0.7426597582037996
discriminator loss: [0.5132294297218323, 0.546875]
adversarial loss: 1.2620880603790283


Train f1-score for act_id  0 improved from  0.749515503875969 to  0.7511531925224569
Validation f1-score for act_id  0 improved from  0.7426597582037996 to  0.7439446366782008
discri